In [1]:
import pandas as pd
import numpy as np
import csv

from datetime import datetime, timedelta

In [2]:
confirmados = pd.read_csv('../data/inputs/Casos_Diarios_Estado_Nacional_Confirmados.csv')
confirmados = pd.melt(confirmados, id_vars=['cve_ent', 'poblacion', 'nombre'], var_name='fecha', value_name='confirmados')
confirmados['fecha'] = pd.to_datetime(confirmados['fecha'], format='%d-%m-%Y')
confirmados = confirmados.drop(columns=['poblacion'])

display(confirmados.head(3))

,cve_ent,nombre,fecha,confirmados
0,1,AGUASCALIENTES,2020-02-26,0
1,2,BAJA CALIFORNIA,2020-02-26,0
2,3,BAJA CALIFORNIA SUR,2020-02-26,0


In [3]:
defunciones = pd.read_csv('../data/inputs/Casos_Diarios_Estado_Nacional_Defunciones.csv')
defunciones = pd.melt(defunciones, id_vars=['cve_ent', 'poblacion', 'nombre'], var_name='fecha', value_name='defunciones')
defunciones['fecha'] = pd.to_datetime(defunciones['fecha'], format='%d-%m-%Y')
defunciones = defunciones.drop(columns=['poblacion'])

display(defunciones.head(3))

,cve_ent,nombre,fecha,defunciones
0,1,AGUASCALIENTES,2020-03-17,0
1,2,BAJA CALIFORNIA,2020-03-17,0
2,3,BAJA CALIFORNIA SUR,2020-03-17,0


In [4]:
negativos = pd.read_csv('../data/inputs/Casos_Diarios_Estado_Nacional_Negativos.csv')
negativos = pd.melt(negativos, id_vars=['cve_ent', 'poblacion', 'nombre'], var_name='fecha', value_name='negativos')
negativos['fecha'] = pd.to_datetime(negativos['fecha'], format='%d-%m-%Y')
negativos = negativos.drop(columns=['poblacion'])

display(negativos.head(3))

,cve_ent,nombre,fecha,negativos
0,1,AGUASCALIENTES,2019-12-31,0
1,2,BAJA CALIFORNIA,2019-12-31,0
2,3,BAJA CALIFORNIA SUR,2019-12-31,0


In [7]:
consolidado = confirmados.merge(negativos, how='outer', on=['cve_ent', 'nombre', 'fecha'])
consolidado = consolidado.merge(defunciones, how='outer', on=['cve_ent', 'nombre', 'fecha'])
consolidado['confirmados'] = consolidado['confirmados'].fillna(0)
consolidado['negativos'] = consolidado['negativos'].fillna(0)
consolidado['defunciones'] = consolidado['defunciones'].fillna(0)

consolidado['casos'] = consolidado['confirmados'] + consolidado['defunciones']
consolidado = consolidado.loc[consolidado['cve_ent'] != 0]
consolidado['pais'] = 'MEXICO'

consolidado = consolidado[['pais', 'nombre', 'fecha', 'confirmados', 'negativos', 'casos', 'defunciones']]
consolidado = consolidado.rename(columns={'nombre': 'estado'})

fecha_max = consolidado['fecha'].max()
fecha_estable = fecha_max + timedelta(days = -7)

print('Fecha máxima original:', fecha_max)
consolidado = consolidado.loc[consolidado['fecha'] <= fecha_estable]
print('Fecha de corte establecida:', consolidado['fecha'].max())

consolidado = consolidado.sort_values(by=['pais', 'estado', 'fecha'])

display(consolidado.head(3))

Fecha máxima original: 2022-02-07 00:00:00
Fecha de corte establecida: 2022-01-31 00:00:00


,pais,estado,fecha,confirmados,negativos,casos,defunciones
23529,MEXICO,AGUASCALIENTES,2019-12-31,0.0,0,0.0,0.0
23562,MEXICO,AGUASCALIENTES,2020-01-01,0.0,0,0.0,0.0
23595,MEXICO,AGUASCALIENTES,2020-01-02,0.0,2,0.0,0.0


In [8]:
consolidado.to_csv('../data/processed/casos_covid.csv', index=False)